*Do not change the cell below - it is for formatting.*

In [ ]:
%%html
<style>
table,td,tr,th {border:none!important}
img {width:400px}
</style>

## Homework 9: Choropleth Maps of US Census Data

This notebook explores the process of creating [choropleth maps](https://en.wikipedia.org/wiki/Choropleth_map) of US Census data. Choropleth maps are maps that symbolize quantitative data values aggregated by regions.  The symbology used for each region indicates the value for the region as a whole, though not necessarily for every location within the region.  For example, a choropleth map of median household income by county may symbolize the areas with a red squential color palette. Higher income areas would be dark red and the lower incomes would be light pink and the range of incomes in between would be mapped to sequential shades of red. Divergent color palettes may also be used if the analyst wished to convey the extremes in the data rather than the trends or average values.


<table>
<tr>
<td>Single color sequential palette</td><td><img src="https://upload.wikimedia.org/wikipedia/commons/a/ae/Color_progression_examples_single_hue.PNG" />
</td>
</tr>

<tr>
<td>Blended color sequential palette</td><td><img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Color_progression_examples_blended_hue.PNG" />
</td>
</tr>
<tr>
<td> Two color divergent palette</td><td>
<img src="https://upload.wikimedia.org/wikipedia/commons/9/90/Color_progression_examples_bi-polar.PNG" />
</td>
</tr>
<tr>
<td></td>
<td>
Images from: wikimedia.org </td>
</tr>
</table>

<br>
Data values are typically binnned into 5 - 7 classes to make the distinctions between values represented on the map more discernable by the human eye. The range of values in each bin depends on the classification method used and the number of classes. Different colors, classification schemes and numbers of classes will result in different data visualizations in a choropleth map. This can can be a useful exploratory data analysis tool.  However, when choropleth maps are used to communicate the data to others, viewers may draw different conclusions depending these choices. Thus, the map maker should choose these values carefully.

Continuous color, or unclassified, choropleth maps are sometimes used to avoid the biases imposed on the data by the choice of classification scheme and number of classes.  While they are useful for displaying trends and outliers in the data, it is hard for human eye to visually interpret data values based from continous colors.

This exercise explores choropleth mapping and some of the issues around classification schemes, classes, and symbology choices. 

In [ ]:
# Import libraries -  run but don't change
from datascience import *   # The basics
import numpy as np
import math, random

import json  # for loading geodata and creating shapely geometries and testing spatial relationships

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## Mapping Poverty

The choropleth maps that we will make are of census data on poverty in San Francisco. We will visualize the US Census American Communnity Survey Table **C17002: Ratio of income to poverty level in the last twelve months**. See the web page [How the Census measures poverty](https://www.census.gov/hhes/www/poverty/about/overview/measure.html) for more information on this and related data tables. 

The ratio of income to poverty table reports the number of people living in a household by the ratio of the household income to the poverty level. For example, the poverty level for a family of four in 2014 was $24,230. A family of four with a household income of 12,115 would have a ratio of 0.5. Another way of saying it would be that those four persons where living 50% below the poverty line.

Let's begin by importing the data from the 2010-2014 ACS table C17002. The tabular data were downloaded from [socialexplorer.com](socialexplorer.com) an online data mapping and reporting website to which UCB has a sitelicense. The data download includes the table of census data in CSV file format and a metadata file, also called a data dicitonary or code book, that describes the columns in the data. The geographic data, in the form of a TIGER/Line shapefile, were downloaded from the [US Cenus website](https://www.census.gov/geo/maps-data/data/tiger-cart-boundary.html).


In [ ]:
# The census ACS 2010-2014 Poverty data files
sfpov_2014_census_table = 'sfpov_2014_R11160245_SL140.csv'
sfpov_2014_census_metadata = 'sfpov_2014_R11160245.txt'
sftracts_2014_geodata = 'sftracts_2014_carto.geojson'

In [ ]:
# Let's take a look at the metadata (or data dictionary, or codebook)
with open (sfpov_2014_census_metadata, "r") as myfile:
    sfpoverty_metadata=myfile.readlines()
sfpoverty_metadata

The metadata isn't pretty in this format but it is essential for understanding the data.  We can see that the column **C17002001** contains the sample estimate of the number of people living in the census tract. The column **C17002002** contains the number of people in the tract supported by an income that is 50% (.5) or more below the poverty line.


### Read in the Data

First, read the census tabular data into our jupyter notebook. 

In [ ]:
# Load the census table data
sfpoverty = Table.read_table(sfpov_2014_census_table)
sfpoverty

Now, load the census tract data, which are in [GeoJSON](geojson.org) format, and take a look at one record so that we can see the structure of the data.

In [ ]:
# Read in the GeoJSON data.
geojson_data = json.load(open(sftracts_2014_geodata))

# print out the properties of one feature
print(geojson_data['features'][1]['properties'])
 

### Map the Census Tracts
We just read in the GeoJSON data to view one of the census tract features that it contains. We now use the **Map.read_geojson** function to read the GeoJSON file directly into a datascience.map object. We can then display the data.

In [ ]:
# Create Map of SF census tracts from the geojson data
# We need to read it in from the file!
sftracts_map = Map.read_geojson(sftracts_2014_geodata)
sftracts_map

## Joing the data


In order to symbolize the census tracts based on the data values we need to join the table data to the geographic data in the map using a **common key**. The key is a column with the same name and values in both data objects.  The closest we have to a match when we load the tables are:
- Census tracts geography: column ** GEOID**
- Census data table: column **Geo_FIPS** or **Geo_GEOID**

The **Geo_FIPS** data values differ slightly from the **GEOID** in that they are missing a leading zero. On the other hand, the **Geo_GEOID** values have more characters than the **GEOID**. Both of these can be cleaned to match the format of the data values in the **GEOID** table. However, we will use **Geo_GEOID** because it is pretty simple to extract the last 11 digits from the string into a new column. This process is shown below.

In [ ]:
# Create a GEOID column to match the geographic data to the table data
# by extracting the last 11 digits from the Geo_GEOID column values
sfpoverty['GEOID'] = sfpoverty.apply(lambda x: x[-11:],['Geo_GEOID'])

# now test
print("The Geo_FIPS value for row one was [%s] but is now GEOID [%s]" % (sfpoverty['Geo_FIPS'][0],sfpoverty['GEOID'][0]))

### FIPS Codes
Now for a brief word about FIPS codes. FIPS stands for federal information processing standards. These standard codes are used to identify different types of data for informaton processing and exchange.  Census FIPS codes are used to uniquely identify different geographic units such as a census tract or blockgroup.  For example:
<img src="http://www.policymap.com/blog/wp-content/uploads/2012/08/FIPSCode_Part4.png" title="Source: policymap.com" />

In the census data we are using the geographic units are tracts so only the 11 digits for the state (2 digits) + county (3 digits) + census tract (6 digits) are included. Unfortunately, because the first digit is often a zero it may be dropped when loaded from a file to a computer program.

### Map the Data Already

Now that we have created that common key, **GEOID**, we can map the data values. First we subset the big table into a new table with only two columns: (1) one of the columns is the GEOID and (2) the other is the data column we wish to map. We then call the **Map.color** method on the sftracts_map to color the map areas by the data vaues.  We show this below by mapping those folks living on an income that is 50% below the poverty level (hard to believe in San Francisco!). We know from the metadata that the counts for each census tract are in the **ACS14_5yr_C17002002** column.

In [ ]:
# Create a two column data table that we can join to the map data
sfpoverty_ct = sfpoverty.select(['GEOID','ACS14_5yr_C17002002'])

# Now map the poor
sftracts_map.color(sfpoverty_ct, key_on='feature.properties.GEOID',line_color="black", 
                   fill_opacity=.75,line_weight=.25)

Mapping those people living on an income 50% below the poverty level doesn't seem to sufficiently capture the number of people struggling financially in San Francisco. Let's create a custom variable to map that includes the count of all people below 1.5 times the poverty level. This is shown below.

In [ ]:
# Create a column with the total number of persons below 1.25 times the poverty level
sfpoverty['poor125'] = sfpoverty['ACS14_5yr_C17002002'] +  \
                            sfpoverty['ACS14_5yr_C17002003'] + \
                            sfpoverty['ACS14_5yr_C17002004']

# Create a two column data table that we can join to the map data
sfpoverty_ct = sfpoverty.select(['GEOID','poor125'])

# Now map the census tracts by our data values
sftracts_map.color(sfpoverty_ct, key_on='feature.properties.GEOID',line_color="black", 
                   fill_opacity=.75,line_weight=.25)

## Mapping Rates

Wow, the above map suggests that there are a lot of people struggling financially in San Francisco. However, we know from our last exercise that mapping raw counts is questionable. Instead of indicating locations with an unusally high number of poor people, maps of raw counts may just be indicating where the population is higher. So let's map the rate - or the percentage of people living in each tract with an income less than 1.25 times below the poverty line.  Use the metadata to see what the column names are that you should use. Also - since we are dividing, we want to remove any census tracts where the population count is zero.

### QUESTION 1
Complete the code below to create the column **percent_poor125** in the **sfpoverty** table.

In [ ]:
## Add your code for QUESTION 1

# First remove census tracts with no population
sfpoverty = sfpoverty.where(sfpoverty['ACS14_5yr_C17002001'] > 0)

# Combine count of poor below 1.24 times the poverty level and divide by the total persons
sfpoverty['percent_poor125'] = 100 * ( (...)  /  sfpoverty['ACS14_5yr_C17002001']) # ADD YOUR CODE HERE

# Create the table to join to the census tracts map
sfpoverty_rate = sfpoverty.select(...)   # ADD YOUR CODE HERE

# Where do most of the poor people live?
sftracts_map.color(...)   # ADD YOUR CODE HERE

### QUESTION 2
Comment on the similarities and differences in the above rate and count maps of people with an income less than 1.25 times the poverty level. Note any factors that make these maps difficult to compare.


**Double-click here to input your answer to question 2.**

### QUESTION 3
In the cell below recreate the poverty_rate map we just made above but give it a custom legend name that indicates more clearly what is being mapped. See the [datascience.maps](http://data8.org/datascience/maps.html) documentation to see how that can be done.

In [ ]:
## Input your code here for QUESTION 3
sftracts_map.color(...)

## Classification Schemes, classes, and colors

In the maps above the color palette, the number of classes into which the data values are binned and the classification scheme used to determine the bin in which an observation falls are all using default values.  In this section we will explore these options and their impact on our mapped data visualizations.

In Homework Exercise 6 we explored classification schemes with graduated symbol and color maps. Choropleth maps are the area data equivilant of point graduated color maps. When using classification schemes the data observations are binned into distinct classes, where 5-7 classes are typical.

The classification method and number of classes greatly impact the visual display of the mapped data. Five classes is often considered the optimal number that can be easily differentiated by the human eye. The selection of classification requires much more thought and is based on the data being visualized. 

We will create and compare maps based on the following three commonly used classification methods:

- **Equal interval** divides observations into N classes with equal data value ranges. For example, if your data values range from 0 to 50 and you have 5 classes, then the data will classed into the following bins: 0-10, 10-20, 20-30, 30-40, 40-50.
    - Works great if your data are well distributed across the full range of values. Also very easy to understand the bin mappings, or class breaks, and create comparisons accross time or data sets when using the same class breaks.
    - Not so great if your data area skewed or have outliers and/or only occupy a few of the classes.
    
    
- **Quantiles** classification places an equal number of observations in each class. For example, if you have 10 observations and 5 classes the data will be ordered by value from low to high and the first two values will go into the first class, etc.
    - Good for creating good looking visualizations as it spreads the data values accross the range of symbology options.
    - Not so great if it creates artificial groupings in your data.


- **Jenks** classification tries to creating groupings that minimize within group variance and maximizes between group variance. Thus, it is called *natural breaks* classification and an *optimal* classifier.
    - Very useful for mapping symbology to data values and giving a sense of the trends in the data.
    - Hard for consumers to understand the rationale for the classes. Difficult to compare two maps as the range of symbols will be different as.
    
    
- **Manual** classification is based on the data scientist's knowledge of the data.
    - Best if you want to compare maps so you can use the same classification scheme.

For a succinct discussion of data classification see: Indiemapper.com's [The Basics of Data Classification](http://indiemapper.com/app/learnmore.php?l=classification).

In [ ]:
## Import some functions to create data class breaks based on different classification methods.
## The classification methods we will try are: equal interval, quantile, and Jenks.
from class_intervals_only import * 

Before mapping the data using different classification schemes it is a good idea to take a look at the range of data values by creating a histogram of sfpoverty['percent_poor125'].

### QUESTION 4.a
In the cell below create a histogram of the data in the **sfpoverty['percent_poor125']** column.

In [ ]:
# Your code to answer Question 4.a


### QUESTION 4.b
What does the histogram tell you about the distribution of poor persons in San Francisco census tracts?

**Double-click here to input your answer to QUESTION 4.B**

### Classifying the data
We have four classification methods that we want to try on the data: manual, equal interval, quantile, and Jenks. We can use the following functions to create the break points, or cuts, that indicate the class intervals. For a manual classification you do not use a method or function but rather define an array of the break points.

- **Equal Interval:** output_array_of_breakpoints = equal(array_of_values)
- **Quantile:** output_array_of_breakpoints = quantile(array_of_values)
- **Jenks:** output_array_of_breakpoints = jenks(array_of values)
- **Manual:** output_array_of_breakpoints = [array_of_values]

For each of the above classification methods the default number of classes is five. We will use that default in the code example below.

In [ ]:
# Example code for creating class breaks
# Create an array of class breaks based on the equal interval classifincation and 5 classes:
mybreaks = equal(sfpoverty_rate['percent_poor125'])
print("Equal breaks: ", mybreaks)


#### Applying a classification to the map
We can then apply these breaks to our map by setting them as the threshod_scale. This is shown below.


In [ ]:
# equal sized data ranges
myscale_equal = equal(sfpoverty_rate['percent_poor125'])
 
sftracts_map.color(sfpoverty_rate, key_on='feature.properties.GEOID', 
                   palette='YlOrRd', 
                   threshold_scale=myscale_equal,
                   line_color="black", 
                   fill_opacity=.75,
                   line_weight=.25)

### Changing the color palette
You can see above that we changed the default color palette of 'YlOrBr', or yellow-orange-brown, to 'YlOrRd', or yellow-orange-red. See the [datascience.maps](http://data8.org/datascience/maps.html) **color** method documentation for available color palettes. You can also see some of the other symbology options you can change like line_color, fill_opacity and line_weight, etc.

### QUESTION 5
In the cell below recreate the previous map using a different **sequential** color palette. Also change the line_weight, line_color and fill_opacity values.

In [ ]:
## Input your code for QUESTION 5 here.

### QUESTION 6
In the cells below create four choropleth maps of the **sfpoverty_rate['percent_poor125']** data using the different classification methods. **IMPORTANT**: keep the color palette and other symbology options the same for all four to facilitate comparison. Also, use the default number of classes. 

In [ ]:
# Input code for your Equal Interval Choropleth Map here
myscale_quantile = ... ## Your code here
 
sftracts_map.color(...)  ## your code here

In [ ]:
# Input code for your Quantile Choropleth Map here
myscale_quantile = quantile(...)
 
sftracts_map.color(...)

In [ ]:
# Input code for your Jenks Choropleth Map here
myscale_jenks = quantile(...)  ## your code here
 
sftracts_map.color(...)  ## your code here

In [ ]:
# Input code for your Manual Choropleth Map here
myscale_manual = []  ## your code here
sftracts_map.color(..)  ## your code here

### QUESTION 7
Which of the four choropleth maps that you just made above do you think best displays the poverty rate and why?

**Double-click here to input your answer to QUESTION 7 **

 

### QUESTION 8
In the cells below create **one** choropleth map of the **ACS 2005-2009 ratio of income to poverty data** that you can compare with the map you identified in your answer to the question above. Your 2005-2009 map should use the same classification method and symbology as the one you identified in QUESTION 7 to facilitate comparison.  The data you need to use for the ACS 2005-2009 map are listed in the cell below. Note, we are mapping the data to the 2014 census tracts. Read the comments in the code cells as they may contain hints or clarify things.

In [ ]:
# The census ACS 2010-2014 Poverty data files
sfpov_2009_census_table = 'sfpov_2009_R11162578_SL140.csv'
sfpov_2009_census_metadata = 'sfpov_2009_R11162578.txt'
sftracts_2009_geodata = 'sftracts_2014_carto.geojson'  # use the 2014 census tracts bc similar to 2009

In [ ]:
# Let's take a look at the metadata (or data dictionary, or codebook)  # no code changes of addtions needed to this cell
with open (sfpov_2009_census_metadata, "r") as myfile:
    sfpoverty09_metadata=myfile.readlines()
sfpoverty09_metadata

In [ ]:
# Read in the cenus table data
sfpoverty09 = ... # your code here
sfpoverty09

In [ ]:
# First remove census tracts with no population
sfpoverty09 = sfpoverty09.where(sfpoverty09['ACS09_5yr_C17002001'] > 0)

# Combine count of poor below 1.25 times the poverty level and divide by the total persons
sfpoverty09['percent_poor125'] = ... # INPUT your code 

In [ ]:
# create the table to join to the geo
# Create a GEOID column to match the geographic data to the table data
# by extracting the last 11 digits from the Geo_GEOID column values
sfpoverty09['GEOID'] =  ... # INPUT your code here to create the join keys
sfpoverty_rate09 = ... # INPUT your code here to shorten the legnth of the table

# now test
print("The Geo_FIPS value for row one was [%s] but is now GEOID [%s]" % \ 
      (sfpoverty09['Geo_FIPS'][0],sfpoverty09['GEOID'][0]))

In [ ]:
# Input code for to classify the data  values and then create the map

# Specifify the class intervals
myscale_manual=[10,20,30,40,50]  # INPUT your code here

# Map the data
sftracts_map.color(...)     # INPUT your code here

In [ ]:
sftracts_map.color(sfpoverty_rate, key_on='feature.properties.GEOID', palette='YlOrRd', 
                   threshold_scale=myscale_manual,line_color="black", 
                   fill_opacity=.75,line_weight=.25)

### QUESTION 9
Compare your ACS 2005-2009 choropleth with the ACS 2010-2014 choropleth map. Comment on any diffferences. Does it seem as though poverty in San Francisco has increased or decreased?

**Double-click here to input your answer to QUESTION 9.**

### QUESTION 10

In the cells below:
- A. Create a histogram of the ACS 2005-2009 data in the **sfpoverty09['percent_poor125']** column. 
- B. Does the histogram conveys the same trend you observe in your map? What information does the map convey that the histogram does not?

In [15]:
## Input your code here for QUESTION 10.A


**Double-click here to input your answer to QUESTION 10.B**

## What to submit and when
Submit your assignment as a Jupyter notebook via bCourses by Tuesday April 19 at **11:59pm.**